In [ ]:
# @title 0. Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

print("✅ เชื่อมต่อเรียบร้อย!")

Mounted at /content/drive
✅ เชื่อมต่อเรียบร้อย!


In [ ]:
# @title 1. Setup Environment & Install
!curl -fsSL https://ollama.com/install.sh | sh
!pip install pandas tqdm

import subprocess
import time
import os

print("Starting Ollama Server...")
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)

MODEL_NAME = "gemma3:1b"
print(f"Downloading model: {MODEL_NAME}...")
!ollama pull {MODEL_NAME}

print(f"✅ Setup Complete! Model {MODEL_NAME} is ready.")

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Starting Ollama Server...

✅ Setup Complete! Model gemma3:1b is ready.


In [ ]:
# @title 2. Configuration
import pandas as pd
import requests
import json
import os
import re
from tqdm import tqdm

TARGET_FILE_NAME = "sentiment_temporal.csv"

INPUT_DIR = "/content/drive/MyDrive/ReaLearn/FY 2025/Interns - NU, Batch 3/AJCC_2026/Chanidapha/Dataset"
OUTPUT_DIR = "/content/drive/MyDrive/ReaLearn/FY 2025/Interns - NU, Batch 3/AJCC_2026/Chanidapha/Result/gemma3:1b_temp1_Topp0.07/sentiment_temporal"
INPUT_COLUMN = "Perturbed"
TARGET_MODEL = "gemma3:1b"
OUTPUT_FILENAME_PREFIX = "Result_gemma3:1b_temp1_"

print(f"✅ Config Saved: เตรียมรันไฟล์ {TARGET_FILE_NAME} ด้วยโมเดล {TARGET_MODEL}")

✅ Config Saved: เตรียมรันไฟล์ sentiment_temporal.csv ด้วยโมเดล gemma3:1b


In [ ]:
# @title 3. Helper Functions

def clean_gemma_response(text):

    if not text: return "Error"

    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL)

    text_lower = text.lower()
    if "positive" in text_lower: return "Positive"
    elif "negative" in text_lower: return "Negative"
    elif "neutral" in text_lower: return "Neutral"
    else: return text.strip()

def create_prompt(sentence):
    system_part = """You are an assistant that classifies the sentiment of the
message into positive, negative, and neutral. Given below is an example
of the sentiment analysis task.

Sentence: I had a bad experience
Sentiment: Negative"""

    user_part = f"""What is the sentiment of the following sentence? Limit your
answer to only one of these options: Positive, Negative, or Neutral.
{sentence}"""

    return [
        {"role": "system", "content": system_part},
        {"role": "user", "content": user_part}
    ]

def query_ollama(model, messages):
    url = "http://localhost:11434/api/chat"
    payload = {
        "model": model,
        "messages": messages,
        "stream": False,
        "options": {"temperature": 1.0,
                    "top_p": 0.07}
    }
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            return response.json()['message']['content']
        else:
            return f"Error: {response.status_code}"
    except Exception as e:
        return f"Error: {e}"

print("✅ ระบบพร้อมทำงาน")

✅ ระบบพร้อมทำงาน


In [ ]:
# @title 4. Start Processing
import datetime
import os
import pandas as pd
from tqdm import tqdm

NUM_RUNS = 3
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
input_path = os.path.join(INPUT_DIR, TARGET_FILE_NAME)

clean_filename = TARGET_FILE_NAME.replace('.csv', '')
target_output_filename = f"{OUTPUT_FILENAME_PREFIX}{clean_filename}_{timestamp}.csv"
output_path = os.path.join(OUTPUT_DIR, target_output_filename)

print(f"🚀 Processing: {TARGET_FILE_NAME}")
print(f"🔄 Mode: 3 Runs (No Vote)")
print(f"📂 Reading from: {input_path}")
print(f"💾 Saving to (New File): {output_path}")

if not os.path.exists(input_path):
    print(f"❌ Error: ไม่พบไฟล์ '{input_path}'")
else:
    try:
        df = pd.read_csv(input_path)

        if INPUT_COLUMN not in df.columns:
            print(f"❌ Error: ไม่พบคอลัมน์ชื่อ '{INPUT_COLUMN}' ในไฟล์ CSV")
        else:
            all_predictions = {f"Prediction{i}": [] for i in range(1, NUM_RUNS + 1)}

            # --- 1. เริ่มวนลูป ---
            for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Generating"):
                sentence = str(row[INPUT_COLUMN])
                prompt_messages = create_prompt(sentence)

                # วนลูปย่อย 3 รอบ
                for i in range(1, NUM_RUNS + 1):
                    raw_res = query_ollama(TARGET_MODEL, prompt_messages)
                    final_pred = clean_gemma_response(raw_res) # ใช้ฟังก์ชัน clean ของ gemma
                    all_predictions[f"Prediction{i}"].append(final_pred)

            ref_col_name = 'Expected_answer'

            if ref_col_name in df.columns:
                insert_loc_start = df.columns.get_loc(ref_col_name) + 1
            else:
                insert_loc_start = len(df.columns)

            for i in range(1, NUM_RUNS + 1):
                col_name = f"Prediction{i}"

                if col_name in df.columns:
                    df.drop(columns=[col_name], inplace=True)

                current_loc = insert_loc_start + (i - 1)

                if current_loc > len(df.columns):
                    df[col_name] = all_predictions[col_name]
                else:
                    df.insert(current_loc, col_name, all_predictions[col_name])

            # --- 3. บันทึกไฟล์ ---
            os.makedirs(OUTPUT_DIR, exist_ok=True)
            df.to_csv(output_path, index=False)
            print(f"\n✅ เสร็จสมบูรณ์! ไฟล์ (3 Predictions) ถูกบันทึกที่: {output_path}")

    except Exception as e:
        print(f"\n❌ Critical Error: {e}")

🚀 Processing: sentiment_temporal.csv
🔄 Mode: 3 Runs (No Vote)
📂 Reading from: /content/drive/MyDrive/ReaLearn/FY 2025/Interns - NU, Batch 3/AJCC_2026/Chanidapha/Dataset/sentiment_temporal.csv
💾 Saving to (New File): /content/drive/MyDrive/ReaLearn/FY 2025/Interns - NU, Batch 3/AJCC_2026/Chanidapha/Result/gemma3:1b_temp1_Topp0.07/sentiment_temporal/Result_gemma3:1b_temp1_sentiment_temporal_20251224_070037.csv


Generating: 100%|██████████| 50/50 [03:34<00:00,  4.28s/it]



✅ เสร็จสมบูรณ์! ไฟล์ (3 Predictions) ถูกบันทึกที่: /content/drive/MyDrive/ReaLearn/FY 2025/Interns - NU, Batch 3/AJCC_2026/Chanidapha/Result/gemma3:1b_temp1_Topp0.07/sentiment_temporal/Result_gemma3:1b_temp1_sentiment_temporal_20251224_070037.csv
